In [ ]:
%pip install -q -r requirements.txt

In [ ]:

import importlib
import functions.core_functions as core_functions
import dask
import dask.dataframe as dd
from dask.dataframe.utils import assert_eq
import pandas as pd
import numpy as np
import pandas_gbq
import json
import gc
import cudf
import os

dask.config.set({"dataframe.backend": "cudf"})

importlib.reload(core_functions)

In [ ]:
resp = {}
resp = core_functions.initialize_clients(service_account_secret_name='SA_ADHOC_BILLING')
resp2 = core_functions.initialize_clients(service_account_secret_name='SA_N90_CORE_APPS')

config = resp.get('config')
bigquery_client = resp.get('clients').get('bigquery_client')
n90_bigquery_client = resp2.get('clients').get('bigquery_client')
storage_client = resp.get('clients').get('storage_client')
sf_client = resp.get('clients').get('sf_client')
veil_billing = resp.get('config').get('veil_billing')
veil_vars = resp.get('config').get('veil_billing').get('vars')
# print(veil_billing)
sfdc_adv_account_cols = veil_billing.get('vars').get('sfdc_adv_account_cols')
sfdc_rate_card_cols = veil_billing.get('vars').get('sfdc_rate_card_cols')
unknown_dma_overrides = config.get('national_dma_overrides_to_us_national')

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_ADHOC_BILLING')

avs_tables = ['detections']
avs_data = core_functions.fetch_table_data(
    project_id=veil_billing.get('avs_project_id'),
    dataset_id=veil_billing.get('avs_dataset_id'),
    table_names=avs_tables,
    bigquery_client=bigquery_client
)

# Access specific DataFrames
detections_df = avs_data['detections']


# encoding_format_ids = encodings_df['format_id'].unique().tolist()

In [ ]:
mongo_tables = ['dmas', 'shows','master_channels']
mongo_data = core_functions.fetch_table_data(
    project_id=veil_billing.get('avs_project_id'),
    dataset_id='mongo',
    table_names=mongo_tables,
    bigquery_client=bigquery_client
)
dmas_df = mongo_data['dmas']
shows_df = mongo_data['shows']
master_channels_df = mongo_data['master_channels']

In [ ]:
billing_tables = ['encodings']
billing_data = core_functions.fetch_table_data(
    project_id=veil_billing.get('billing_project_id'),
    dataset_id=veil_billing.get('billing_dataset_id'),
    table_names=billing_tables,
    bigquery_client=bigquery_client
)

# Access specific DataFrames
encodings_df = billing_data['encodings']

In [ ]:
encodings_df.sort_values(by=['encoding_id', 'billing_last_updated'], ascending=[True, False], inplace=True)
encodings_df.drop_duplicates(subset=['encoding_id'], keep='first', inplace=True)

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_N90_CORE_APPS')
geo_sql = f"""
    SELECT * from `next90-core-applications.next90_analytics.geos` WHERE geo_type = 'dma'
"""
geo_df = core_functions.fetch_gbq_data(query=geo_sql, bigquery_client=n90_bigquery_client)
# n90_bigquery_client

In [ ]:


# # small_detections_df = detections_df.head(1000)
# # small_detections_df['date_time'] = pd.to_datetime(small_detections_df['date_time'], utc=True)
# detections_df['date_time'] = pd.to_datetime(detections_df['date_time'], utc=True)
# detections_df['year'] = detections_df['date_time'].dt.year
# detections_df['month'] = detections_df['date_time'].dt.month
# detections_df['day'] = detections_df['date_time'].dt.day
# detections_df['tv_show_id'].fillna(-10, inplace=True) 
# detections_df['group_occurrence_id'].fillna(-6, inplace=True)
# dmas_df['dma_rank'].replace('', 0).fillna(0, inplace=True)

# df_1 = detections_df.copy()

# # small_detections_df['occurrence_id'] = small_detections_df['occurrence_id'].astype(float)
# df_1 = df_1.merge(master_channels_df, how='left', left_on='broadcaster_id', right_on='broadcaster_id')
# df_1 = df_1.merge(dmas_df, how='left', left_on='dma_id', right_on='dma_id')
# shows_df['show_id'].fillna(-5, inplace=True)
# df_1 = df_1.merge(shows_df, how='left', left_on='tv_show_id', right_on='show_id')


mask = ((df_1['dma_id'].isin(unknown_dma_overrides)) )
df_1.loc[mask, 'neustar_id'] = 808080
df_1['neustar_id'] = df_1['neustar_id'].fillna(808080)
df_1['neustar_id'] = df_1['neustar_id'].astype(int)
df_2 = df_1.copy()
geo_df['geo_location'] = geo_df['geo_location'].astype(int)
df_2 = df_2.merge(geo_df, how='left', left_on='neustar_id', right_on='geo_location')
df_1 = df_1.merge(encodings_df[['encoding_id', 'format_id', 'customer_id', 'sfdc_account_id', 'sfdc_advertiser_id', 'aeis_id']], how='left', left_on='encoding_id', right_on='encoding_id')
df_2 = df_2.merge(encodings_df, how='left', left_on='encoding_id', right_on='encoding_id', suffixes=('', '_encoding'))
billing_last_updated = pd.Timestamp.utcnow()
df_1['billing_det_last_updated'] = billing_last_updated
df_1['billing_det_last_updated'] = pd.to_datetime(df_1['billing_det_last_updated'], utc=True)
df_2['billing_det_last_updated'] = billing_last_updated
df_2['billing_det_last_updated'] = pd.to_datetime(df_2['billing_det_last_updated'], utc=True)
billing_last_audit_id = core_functions.generate_uuid()
df_1['billing_det_last_audit_id'] = billing_last_audit_id
df_2['billing_det_last_updated'] = billing_last_audit_id
df_1.head()

In [ ]:
df_2.loc[df_2['dma_rank'].isna(), 'dma_rank'] 

In [ ]:
for col in df_2.columns:
    print(f"{col}: {df_2[col].dtype}")

In [ ]:
df_1.sort_values(by=['sfdc_account_id', 'year', 'month', 'day'], inplace=True)
df_2.sort_values(by=['sfdc_account_id', 'year', 'month', 'day'], inplace=True)
df_1_valid_final_cols = ['occurrence_id',
 'encoding_id',
 'broadcaster_id',
 'cost',
 'tv_show_id',
 'origin',
 'group_occurrence_id',
 'last_updated',
 'last_audit_id',
 'date_time',
 'year',
 'month',
 'day',
 'channel_id',
 'affiliate',
 'callsign',
 'dma_id',
 'kantar_region',
 'neustar_id',
 'name',
 'timezone',
 'dma_rank',
 'id',
 'show_id',
 'is_barter',
 'is_cable',
 'is_network',
 'is_spot',
 'is_canadian',
 'is_hispanic',
 'is_local_cable',
 'is_active',
 'length',
 'show_name',
 'format_id',
 'customer_id',
 'sfdc_account_id',
 'sfdc_advertiser_id',
 'aeis_id',
 'billing_det_last_updated',
 'billing_det_last_audit_id']

df_2_valid_final_cols = ['occurrence_id',
 'encoding_id',
 'broadcaster_id',
 'cost',
 'tv_show_id',
 'origin',
 'group_occurrence_id',
 'last_updated',
 'last_audit_id',
 'date_time',
 'year',
 'month',
 'day',
 'channel_id',
 'affiliate',
 'callsign',
 'dma_id',
 'kantar_region',
 'neustar_id',
 'name',
 'timezone',
 'dma_rank',
 'id',
 'show_id',
 'is_barter',
 'is_cable',
 'is_network',
 'is_spot',
 'is_canadian',
 'is_hispanic',
 'is_local_cable',
 'is_active',
 'length',
 'show_name',
 'geo_location',
 'geo_type',
 'geo_latitude',
 'geo_longitude',
 'geo_city',
 'geo_state',
 'geo_country',
 'geo_neustar_id',
 'geo_us_msa_id',
 'geo_us_county_fips_id',
 'geo_ca_cma_id',
 'geo_gm_zip_code',
 'format_id',
 'encoder_group_id',
 'encoded_timestamp',
 'clone_of',
 'status',
 'last_updated_encoding',
 'last_audit_id_encoding',
 'encoder_id',
 'detection_end_date',
 'encoded_timestamp_epoch',
 'attributes_advertiser',
 'attributes_audience',
 'attributes_audience_2',
 'attributes_cable_estimate',
 'attributes_campaign',
 'attributes_category',
 'attributes_client_code',
 'attributes_commercial_id',
 'attributes_contour_id',
 'attributes_creative_offer',
 'attributes_description',
 'attributes_donovan_agency_advertiser_code',
 'attributes_donovan_agency_estimate_code',
 'attributes_donovan_agency_product_code',
 'attributes_eid',
 'attributes_group',
 'attributes_hd_sd',
 'attributes_id',
 'attributes_isci',
 'length_in_seconds',
 'attributes_length',
 'attributes_lob',
 'attributes_media_type',
 'attributes_message',
 'attributes_misc',
 'attributes_module_code',
 'attributes_offer',
 'attributes_offer_2',
 'attributes_phone_number',
 'attributes_product_code',
 'attributes_product_name',
 'attributes_project_name',
 'attributes_quality',
 'attributes_revision',
 'attributes_show_name',
 'attributes_slug',
 'attributes_sport_id',
 'attributes_sport_show_sub_category',
 'attributes_spot_estimate',
 'attributes_spot_name',
 'attributes_tag',
 'attributes_text',
 'attributes_title',
 'attributes_veil_id',
 'attributes_version_name',
 'attributes_year',
 'product_code',
 'isci',
 'advertiser',
 'encoder_group_name',
 'encoder_group__deleted',
 'encoder_group__last_audit_id',
 'encoder_group__last_updated',
 'aeis_id',
 'aeis__encoding_id',
 'aeis__encoding_offset',
 'aeis__last_updated',
 'aeis__last_audit_id',
 'format_name',
 'format__profile_id',
 'format__customer_id',
 'format__report_breakup',
 'format__deleted',
 'format__last_updated',
 'format__last_audit_id',
 'sfdc_account_name',
 'customer_id',
 'account_id',
 'contract_item',
 'customer_name',
 'contract_number',
 'sales_person_code',
 'deleted',
 'profile_id',
 'profile_name',
 'profile__deleted',
 'profile__default_asset_code',
 'profile__last_updated',
 'profile__last_audit_id',
 'ad_prod_campaign',
 'campaign',
 'sfdc_advertiser_id',
 'billing_last_updated',
 'billing_last_audit_id',
 'sfdc_account_id',
 'billing_det_last_updated']


In [ ]:
df_1_date_cols = ['date_time', 'last_updated','billing_det_last_updated']
df_1_int_cols = ['occurrence_id', 'encoding_id', 'broadcaster_id', 'tv_show_id', 'group_occurrence_id', 'year', 'month', 'day', 'neustar_id', 'dma_rank', 'show_id', 'format_id', 'customer_id', 'aeis_id']
df_1_float_cols = ['cost', 'length']
df_1_bool_cols = ['is_barter', 'is_cable', 'is_network', 'is_spot', 'is_canadian', 'is_hispanic', 'is_local_cable', 'is_active']


df_2_date_cols = ['date_time', 'last_updated','billing_det_last_updated', 'encoded_timestamp','last_updated_encoding','detection_end_date','encoder_group__last_updated','aeis__last_updated',
             'format__last_updated','profile__last_updated','billing_last_updated']
df_2_int_cols = ['occurrence_id', 'encoding_id', 'broadcaster_id', 'tv_show_id', 'group_occurrence_id', 'year', 'month', 'day', 'neustar_id', 'dma_rank', 'show_id', 'format_id', 'customer_id', 'aeis_id',
            'encoder_group_id', 'clone_of', 'encoder_id','encoded_timestamp_epoch','aeis__encoding_id','format__profile_id','format__customer_id',
            'profile_id']
df_2_float_cols = ['cost', 'length', 'geo_latitude', 'geo_longitude']
df_2_bool_cols = ['is_barter', 'is_cable', 'is_network', 'is_spot', 'is_canadian', 'is_hispanic', 'is_local_cable', 'is_active','encoder_group__deleted',
             'format__deleted','deleted','profile__deleted']

In [ ]:
# for col in df_2.columns:
#     print(f"'{col}': '{df_2[col].dtype}',")

In [ ]:
df_1_schema = {
    'occurrence_id': 'int64',
    'encoding_id': 'int64',
    'broadcaster_id': 'int64',
    'cost': 'float64',
    'tv_show_id': 'int64',
    'origin': 'string',
    'group_occurrence_id': 'int64',
    'last_updated': 'datetime64[ns, UTC]',
    'last_audit_id': 'string',
    'date_time': 'datetime64[ns, UTC]',
    'year': 'int64',
    'month': 'int64',
    'day': 'int64',
    'channel_id': 'string',
    'affiliate': 'string',
    'callsign': 'string',
    'dma_id': 'string',
    'kantar_region': 'string',
    'neustar_id': 'int64',
    'name': 'string',
    'timezone': 'string',
    'dma_rank': 'int64',
    'id': 'string',
    'show_id': 'int64',
    'is_barter': 'bool',
    'is_cable': 'bool',
    'is_network': 'bool',
    'is_spot': 'bool',
    'is_canadian': 'bool',
    'is_hispanic': 'bool',
    'is_local_cable': 'bool',
    'is_active': 'bool',
    'length': 'float64',
    'show_name': 'string',
    'format_id': 'int64',
    'customer_id': 'int64',
    'sfdc_account_id': 'string',
    'sfdc_advertiser_id': 'string',
    'aeis_id': 'int64',
    'billing_det_last_updated': 'datetime64[ns, UTC]',
    'billing_det_last_audit_id': 'string'
}

df_2_schema = {
   'occurrence_id': 'int64',
   'encoding_id': 'int64',
   'broadcaster_id': 'int64',
   'cost': 'float64',
   'tv_show_id': 'int64',
   'origin': 'string',
   'group_occurrence_id': 'int64',
   'last_updated': 'datetime64[ns, UTC]',
   'last_audit_id': 'string',
   'date_time': 'datetime64[ns, UTC]',
   'year': 'int64',
   'month': 'int64',
   'day': 'int64',
   'channel_id': 'string',
   'affiliate': 'string',
   'callsign': 'string',
   'dma_id': 'string',
   'kantar_region': 'string',
   'neustar_id': 'int64',
   'name': 'string',
   'timezone': 'string',
   'dma_rank': 'int64',
   'id': 'string',
   'show_id': 'int64',
   'is_barter': 'bool',
   'is_cable': 'bool',
   'is_network': 'bool',
   'is_spot': 'bool',
   'is_canadian': 'bool',
   'is_hispanic': 'bool',
   'is_local_cable': 'bool',
   'is_active': 'bool',
   'length': 'float64',
   'show_name': 'string',
   'geo_location': 'string',
   'geo_type': 'string',
   'geo_latitude': 'float64',
   'geo_longitude': 'float64',
   'geo_city': 'string',
   'geo_state': 'string',
   'geo_country': 'string',
   'geo_neustar_id': 'string',
   'geo_us_msa_id': 'string',
   'geo_us_county_fips_id': 'string',
   'geo_ca_cma_id': 'string',
   'geo_gm_zip_code': 'string',
   'format_id': 'int64',
   'encoder_group_id': 'int64',
   'encoded_timestamp': 'datetime64[ns, UTC]',
   'clone_of': 'int64',
   'status': 'string',
   'last_updated_encoding': 'datetime64[ns, UTC]',
   'last_audit_id_encoding': 'string',
   'encoder_id': 'int64',
   'detection_end_date': 'datetime64[ns, UTC]',
   'encoded_timestamp_epoch': 'int64',
   'attributes_advertiser': 'string',
   'attributes_audience': 'string',
   'attributes_audience_2': 'string',
   'attributes_cable_estimate': 'string',
   'attributes_campaign': 'string',
   'attributes_category': 'string',
   'attributes_client_code': 'string',
   'attributes_commercial_id': 'string',
   'attributes_contour_id': 'string',
   'attributes_creative_offer': 'string',
   'attributes_description': 'string',
   'attributes_donovan_agency_advertiser_code': 'string',
   'attributes_donovan_agency_estimate_code': 'string',
   'attributes_donovan_agency_product_code': 'string',
   'attributes_eid': 'string',
   'attributes_group': 'string',
   'attributes_hd_sd': 'string',
   'attributes_id': 'string',
   'attributes_isci': 'string',
   'length_in_seconds': 'string',
   'attributes_length': 'string',
   'attributes_lob': 'string',
   'attributes_media_type': 'string',
   'attributes_message': 'string',
   'attributes_misc': 'string',
   'attributes_module_code': 'string',
   'attributes_offer': 'string',
   'attributes_offer_2': 'string',
   'attributes_phone_number': 'string',
   'attributes_product_code': 'string',
   'attributes_product_name': 'string',
   'attributes_project_name': 'string',
   'attributes_quality': 'string',
   'attributes_revision': 'string',
   'attributes_show_name': 'string',
   'attributes_slug': 'string',
   'attributes_sport_id': 'string',
   'attributes_sport_show_sub_category': 'string',
   'attributes_spot_estimate': 'string',
   'attributes_spot_name': 'string',
   'attributes_tag': 'string',
   'attributes_text': 'string',
   'attributes_title': 'string',
   'attributes_veil_id': 'string',
   'attributes_version_name': 'string',
   'attributes_year': 'string',
   'product_code': 'string',
   'isci': 'string',
   'advertiser': 'string',
   'encoder_group_name': 'string',
   'encoder_group__deleted': 'bool',
   'encoder_group__last_audit_id': 'string',
   'encoder_group__last_updated': 'datetime64[ns, UTC]',
   'aeis_id': 'int64',
   'aeis__encoding_id': 'int64',
   'aeis__encoding_offset': 'string',
   'aeis__last_updated': 'datetime64[ns, UTC]',
   'aeis__last_audit_id': 'string',
   'format_name': 'string',
   'format__profile_id': 'int64',
   'format__customer_id': 'int64',
   'format__report_breakup': 'string',
   'format__deleted': 'bool',
   'format__last_updated': 'datetime64[ns, UTC]',
   'format__last_audit_id': 'string',
   'sfdc_account_name': 'string',
   'customer_id': 'int64',
   'account_id': 'string',
   'contract_item': 'string',
   'customer_name': 'string',
   'contract_number': 'string',
   'sales_person_code': 'string',
   'deleted': 'bool',
   'profile_id': 'int64',
   'profile_name': 'string',
   'profile__deleted': 'bool',
   'profile__default_asset_code': 'string',
   'profile__last_updated': 'datetime64[ns, UTC]',
   'profile__last_audit_id': 'string',
   'ad_prod_campaign': 'string',
   'campaign': 'string',
   'sfdc_advertiser_id': 'string',
   'billing_last_updated': 'datetime64[ns, UTC]',
   'billing_last_audit_id': 'string',
   'sfdc_account_id': 'string',
   'billing_det_last_updated': 'datetime64[ns, UTC]'
}

In [ ]:
# Enforce schema

def enforce_schema(df, schema):
    for column, dtype in schema.items():
        if dtype == 'string':
            df[column] = df[column].astype(str).fillna('')
        elif dtype == 'int64':
            df[column] = pd.to_numeric(df[column], errors='coerce').fillna(-1).astype('int64')
        elif dtype == 'float64':
            df[column] = pd.to_numeric(df[column], errors='coerce').fillna(-1.0).astype('float64')
        elif dtype == 'datetime64[ns, UTC]':
            df[column] = pd.to_datetime(df[column], errors='coerce', utc=True).fillna(pd.Timestamp.min.tz_localize('UTC'))
    return df

df_1 = enforce_schema(df_1, df_1_schema)
df_2 = enforce_schema(df_2, df_2_schema)
# for column, dtype in df_1_schema.items():
#     if dtype == 'string':
#         df_1[column] = df_1[column].astype(str).fillna('')
#     elif dtype == 'int64':
#         df_1[column] = pd.to_numeric(df_1[column], errors='coerce').fillna(-1).astype('int64')
#     elif dtype == 'float64':
#         df_1[column] = pd.to_numeric(df_1[column], errors='coerce').fillna(-1.0).astype('float64')
#     elif dtype == 'datetime64[ns, UTC]':
#         df_1[column] = pd.to_datetime(df_1[column], errors='coerce', utc=True).fillna(pd.Timestamp.min.tz_localize('UTC'))

# Validate column types
print(df_1.dtypes)

In [ ]:
# df_1 = core_functions.prep_columns_for_parquet(df_1, df_1_valid_final_cols, date_cols=df_1_date_cols, int_cols=df_1_int_cols, float_cols=df_1_float_cols, bool_cols=df_1_bool_cols)
# print('--------------')

# df_2 = core_functions.prep_columns_for_parquet(df_2, df_2_valid_final_cols, date_cols=df_2_date_cols,  int_cols=df_2_int_cols, float_cols=df_2_float_cols, bool_cols=df_2_bool_cols)


In [ ]:

veil_storage_options = None
veil_storage_options = config.get('VEIL_GCS_STORAGE_OPTIONS')

# n90_storage_options = None
n90_storage_options = config.get('N90_GCS_STORAGE_OPTIONS')

veil_billing_bucket = None
veil_billing_bucket = config.get('veil_billing').get('billing_gcs_bucket_id')

# process_df['profile__attributes']
n90_bucket = None
n90_bucket = 'n90_veil_partner'
veil_output_prefix = None
veil_output_prefix = 'detections'
n90_output_prefix = None
n90_output_prefix = 'advocado-looker/avs_prod/detections'
partition_cols = None
partition_cols = ['sfdc_account_id', 'year', 'month', 'day']
importlib.reload(core_functions)

# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_ADHOC_BILLING')
# core_functions.write_hive_partitioned_parquet(df_1, veil_billing_bucket, veil_output_prefix, partition_cols, veil_storage_options)
# print(f"Finished writing to {veil_billing_bucket}/{veil_output_prefix}")
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config.get('SA_N90_CORE_APPS')
core_functions.write_hive_partitioned_parquet(df_2, n90_bucket, n90_output_prefix, partition_cols, n90_storage_options)
print(f"Finished writing to {n90_bucket}/{n90_output_prefix}")

# new cell
partition_cols
# print(encodings_bvs_df_to_write.dtypes)


In [ ]:

print("GOOGLE_APPLICATION_CREDENTIALS:", os.environ['GOOGLE_APPLICATION_CREDENTIALS'])